In [22]:
import web3
from web3 import Web3

# Define the ABI
abi = [
    {
        "inputs": [],
        "stateMutability": "nonpayable",
        "type": "constructor"
    },
    {
        "anonymous": False,
        "inputs": [
            {
                "indexed": False,
                "internalType": "string",
                "name": "key",
                "type": "string"
            },
            {
                "indexed": False,
                "internalType": "uint128",
                "name": "value",
                "type": "uint128"
            },
            {
                "indexed": False,
                "internalType": "uint128",
                "name": "timestamp",
                "type": "uint128"
            }
        ],
        "name": "OracleUpdate",
        "type": "event"
    },
    {
        "anonymous": False,
        "inputs": [
            {
                "indexed": False,
                "internalType": "address",
                "name": "newUpdater",
                "type": "address"
            }
        ],
        "name": "UpdaterAddressChange",
        "type": "event"
    },
    {
        "inputs": [
            {
                "internalType": "string",
                "name": "key",
                "type": "string"
            }
        ],
        "name": "getValue",
        "outputs": [
            {
                "internalType": "uint128",
                "name": "",
                "type": "uint128"
            },
            {
                "internalType": "uint128",
                "name": "",
                "type": "uint128"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "string[]",
                "name": "keys",
                "type": "string[]"
            },
            {
                "internalType": "uint256[]",
                "name": "compressedValues",
                "type": "uint256[]"
            }
        ],
        "name": "setMultipleValues",
        "outputs": [],
        "stateMutability": "nonpayable",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "string",
                "name": "key",
                "type": "string"
            },
            {
                "internalType": "uint128",
                "name": "value",
                "type": "uint128"
            },
            {
                "internalType": "uint128",
                "name": "timestamp",
                "type": "uint128"
            }
        ],
        "name": "setValue",
        "outputs": [],
        "stateMutability": "nonpayable",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "address",
                "name": "newOracleUpdaterAddress",
                "type": "address"
            }
        ],
        "name": "updateOracleUpdaterAddress",
        "outputs": [],
        "stateMutability": "nonpayable",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "string",
                "name": "",
                "type": "string"
            }
        ],
        "name": "values",
        "outputs": [
            {
                "internalType": "uint256",
                "name": "",
                "type": "uint256"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    }
]

# Define the log data
log_data = {
    "data": "0x00000000000000000000000000000000000000000000000000000000000000600000000000000000000000000000000000000000000000000000053126afd5c6000000000000000000000000000000000000000000000000000000006690e3f70000000000000000000000000000000000000000000000000000000000000008574254432f555344000000000000000000000000000000000000000000000000",
    "topics": [
      "0xa7fc99ed7617309ee23f63ae90196a1e490d362e6f6a547a59bc809ee2291782"
    ],
    "address": "0xdee629af973ebf5bf261ace12ffd1900ac715f5e"
  }





In [23]:
from web3 import Web3

def decode_log_with_abi(log_data: dict, contract_abi: list):
    w3 = Web3()

    # Normalize log's event signature hash
    raw_log_hash = log_data["topics"][0].lower()
    log_event_hash = raw_log_hash[2:] if raw_log_hash.startswith("0x") else raw_log_hash

    # Find matching event with normalized comparison
    matching_event = None
    for item in contract_abi:
        if item.get("type") == "event":
            # Build event signature
            inputs = [inp["type"] for inp in item["inputs"]]
            signature = f"{item['name']}({','.join(inputs)})"

            # Compute and normalize our hash
            computed_raw = w3.keccak(text=signature).hex().lower()
            computed_hash = computed_raw[2:] if computed_raw.startswith("0x") else computed_raw

            if computed_hash == log_event_hash:
                matching_event = item
                break

    if not matching_event:
        available = [e["name"] for e in contract_abi if e.get("type") == "event"]
        raise ValueError(
            f"No event matches normalized hash {log_event_hash}\n"
            f"Available events: {available}"
        )

    # Create dummy log entry with required fields
    log_entry = {
        "topics": [bytes.fromhex(t[2:]) for t in log_data["topics"]],
        "data": bytes.fromhex(log_data["data"][2:]),
        "logIndex": 0,
        "transactionIndex": 0,
        "transactionHash": b'\x00' * 32,
        "address": None,
        "blockHash": b'\x00' * 32,
        "blockNumber": 0
    }

    # Decode using the matched event
    contract = w3.eth.contract(abi=[matching_event])
    event_class = getattr(contract.events, matching_event["name"])
    decoded = event_class().process_log(log_entry)

    return {
        "event_name": matching_event["name"],
        "args": dict(decoded["args"]),
        "raw_log": decoded
    }

In [24]:


# Usage
decoded = decode_log_with_abi(log_data, abi)
print(f"Decoded {decoded['event_name']}:")
for arg, value in decoded["args"].items():
    print(f"{arg}: {value}")

Decoded OracleUpdate:
key: WBTC/USD
value: 5708660594118
timestamp: 1720771575
